In [1]:
import tensorflow as tf

In [2]:
import os,timeit
import numpy as np

In [11]:
from numpy import random

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
from sys import platform
DATA_DIRECTORY = ""
SLASH = ""
if platform == "linux" or platform == "linux2":
    DATA_DIRECTORY = "/home/nitintyagi/wheat data/BULK/"
    SLASH = "/"
elif platform == "win32":
    DATA_DIRECTORY = "D:\mvl\wheat\data\BULK\\"
    SLASH="\\"

In [7]:
MAX_VARIETIES = len(os.listdir(DATA_DIRECTORY)) - 8

In [9]:
#Constants
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.50
TOTAL_IMAGE_COUNT = 8
IMAGE_COUNT = int(TOTAL_IMAGE_COUNT/4)
NUM_VARIETIES = 4
NUM_OF_BANDS = 15
FIRST_BAND = 21
LAST_BAND = 149

IMAGE_WIDTH = 30
IMAGE_HEIGHT = 30

In [13]:
CHOSEN_VARIETIES = random.randint(MAX_VARIETIES, size=(NUM_VARIETIES))
CHOSEN_VARIETIES.sort()
print(CHOSEN_VARIETIES)

[ 3  8 13 14]


In [17]:
CHOSEN_VARIETIES = random.randint(4, size=(2))
CHOSEN_VARIETIES.sort()
print(CHOSEN_VARIETIES)

[1 2]


In [18]:
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [19]:
def dataset_file_name():
    return "./dataset/V"+str(NUM_VARIETIES).zfill(3)+"_IC_"+str(TOTAL_IMAGE_COUNT).zfill(5)+"_FilledArea_"+str(FILLED_AREA_RATIO)+"_NumOfBands_"+str(NUM_OF_BANDS)+"_FB_"+str(FIRST_BAND)+"_LB_"+str(LAST_BAND)+"_BandNo_"+str(BAND_NUMBER)+"_ImageHeight_"+str(IMAGE_HEIGHT)+"_ImageWidth_"+str(IMAGE_WIDTH)

In [20]:
DATASET_FILE_NAME = dataset_file_name()

x_train_dataset = np.load(DATASET_FILE_NAME+"_train_dataset.npy")
y_train_dataset_label = np.load(DATASET_FILE_NAME+"_train_dataset_label.npy")
x_test_dataset = np.load(DATASET_FILE_NAME+"_test_dataset.npy")
y_test_dataset_label = np.load(DATASET_FILE_NAME+"_test_dataset_label.npy")

In [23]:
#filter dataset
def filter_dataset(input,input_label,filter):
    out = []
    out_label = []
    for index,label in enumerate(input_label):
        if label in filter:
            out_label.append(input_label[index])
            out.append(input[index])
            
    return np.array(out),np.array(out_label)

In [24]:
train_dataset, train_dataset_label = filter_dataset(x_train_dataset,y_train_dataset_label,CHOSEN_VARIETIES)
test_dataset, test_dataset_label = filter_dataset(x_test_dataset,y_test_dataset_label,CHOSEN_VARIETIES)

In [26]:
test_dataset_label

array([1, 1, 1, 1, 2, 2, 2, 2])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
nsamples, nx, ny,nb = train_dataset.shape
d2_train_dataset = train_dataset.reshape((nsamples,nx*ny*nb))
d2_test_dataset = test_dataset.reshape((nsamples,nx*ny*nb))

In [ ]:
# creating a RF classifier
clf = RandomForestClassifier(n_estimators = 100)

In [ ]:
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
tic = start_timer()
clf.fit(d2_train_dataset, train_dataset_label)
toc = end_timer()
show_time(tic,toc)

In [ ]:
# performing predictions on the test dataset
y_pred = clf.predict(d2_test_dataset)

print("ACCURACY OF THE MODEL: ", accuracy_score(test_dataset_label, y_pred))

In [ ]:
print(confusion_matrix(test_dataset_label, y_pred))